In [15]:
from qiskit import *
from numpy.random import randint, shuffle
from qiskit.visualization import plot_histogram, plot_bloch_multivector
import numpy as np
%run BB84_functions.ipynb

In [16]:
n = 8
N = 32

In [17]:
#Step 1 - Ram generates bits
ram_bits = bit_string(N)

#Step 2 - Ram generates the bases string
ram_bases = bit_string(N)

#Step 3 - Sita generates the bases string
sita_bases = bit_string(N)

|Information|Dev|Kaal|Abhi|
|:-:|:-:|:-:|:-:|
|Dev's bits|Y|N|N|
|Dev's bases|Y|N|N|
|Abhi's bases|N|Y|N|

In [18]:
#Step 4 - Ram encodes his bits in the chosen bases 
ram_encode = encode_bits(ram_bits, ram_bases)

#Step 5 - Ram sends his encoded bits to Sita and Ravan tries to intercept over the Quantum channel
received_encode = QCC(ram_encode)

#Step 6 - Sita measures Ram's bits in the chosen bases
sita_circuit, sita_bits = measure_bits(received_encode, sita_bases)

#Step 7 - Sita announces that she has received the encoding and measured it over CAC


#Step 8 - Ram and Sita exchange their bases over CAC
r2s = CAC(ram_bases)
s2r = CAC(sita_bases)

#Step 9 - Ram and Sita discard all the bits that correspond to disagreed bases
agreed_base_indices = agreed_bases(ram_bases, sita_bases)
#sita_circuit.draw(output = 'mpl')

|Information|Dev|Kaal|Abhi|
|:-:|:-:|:-:|:-:|
|Dev's Encoding|Y|Partial|Noisy|
|Abhi's bits|N|N|Y|
|Dev's and Abhi's bases|Y|Y|Y|

In [19]:
#Step 10 - Error rate checking 
S2T = bit_string(len(agreed_base_indices))
T = select_bits(agreed_base_indices, S2T, 0)

#Step 11 - Ram tells T to Sita over CAC
sita_T = CAC(T)

#Step 12 - Ram and Sita generate their test bits 
ram_test_bits, sita_test_bits = [], []
for i in T :
    ram_test_bits.append(ram_bits[i])
    sita_test_bits.append(sita_bits[i])

#Step 12 - Ram and Sita announce their test bits to each other over the CAC

#Step 13 - Ram and Sita compute the error rate
error = error_rate(ram_test_bits, sita_test_bits)
print(error)

0.0


|Information|Dev|Kaal|Abhi|
|:-:|:-:|:-:|:-:|
|T|Y|Y|Y|
|Dev's test bits|Y|Y|Y|
|Abhi's test bits|Y|Y|Y|

In [21]:
#Step 14 - Ram and Sita check over a threshold for error before proceeding ahead
if error > 0.11 :
    print ("Abort")
else :
    #Step 15 - Ram and Sita generate their pseudo keys
    SminusT = select_bits(agreed_base_indices, S2T, 1)
    ram_pseudokey, sita_pseudokey = [], []
    for i in SminusT :
        ram_pseudokey.append(ram_bits[i])
        sita_pseudokey.append(sita_bits[i])

    #reconciliation not required

    #Step 17 - Ram and Sita perform privacy amplification
    n, k = len(ram_pseudokey), len(ram_pseudokey) // 2
    seed = bit_string(n + k - 1)
    ram_key = toeplitz(n, k, ram_pseudokey, seed)
    sita_key = toeplitz(n, k, sita_pseudokey, seed)
    print(ram_key, sita_key, error_rate(ram_test_bits, sita_test_bits))

[0, 1, 1, 0] [0, 1, 1, 0] 0.0


|Information|Dev|Kaal|Abhi|
|:-:|:-:|:-:|:-:|
|Pseudo key|Y|Partial|Noisy|